In [1]:
import os
import csv
import sys
import numpy as np
import matplotlib.pyplot as plot
from scipy.stats import poisson
from scipy.stats import gamma
from scipy.stats import beta
from scipy.stats import norm
import pandas as pd
import itertools
from scipy.stats import dirichlet
%run -i utils.py
sys.path.insert(0, '../')
from get_team_api import *
from get_api import *

### Optimization of players over any fixtures

In [2]:
all_players_params = pd.read_csv("../parameters/all_players_params.csv")
all_teams_params = pd.read_csv("../parameters/all_teams_params.csv", header=None)
teams = pd.read_csv("../parameters/all_teams.csv", header=None).values[:, 0]

In [5]:
# fixture list this season to predict
fixture_list_this_season = getFixtures(np.linspace(9, 13, 5).astype(int), team_id_file="../data/team_id_20192020.csv",
                                       filelead="../")

In [6]:
C, S, CS, G, MP, KB = ComputeExpectedPoints(fixture_list_this_season, teams, all_players_params, all_teams_params, Niter=500,
                                        additionalstats=True)

utils.py:382: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  intercept = (all_teams_params.as_matrix())[0, :]
utils.py:383: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  mu = (all_teams_params.as_matrix())[1, :]
utils.py:384: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  a = (all_teams_params.as_matrix())[2:(len(teams) + 2), :]
utils.py:385: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  d = (all_teams_params.as_matrix())[(len(teams) + 2):((2 * len(teams)) + 2), :]


---
Realisation  0
---
Realisation  1
---
Realisation  2
---
Realisation  3
---
Realisation  4
---
Realisation  5
---
Realisation  6
---
Realisation  7
---
Realisation  8
---
Realisation  9
---
Realisation  10
---
Realisation  11
---
Realisation  12
---
Realisation  13
---
Realisation  14
---
Realisation  15
---
Realisation  16
---
Realisation  17
---
Realisation  18
---
Realisation  19
---
Realisation  20
---
Realisation  21
---
Realisation  22
---
Realisation  23
---
Realisation  24
---
Realisation  25
---
Realisation  26
---
Realisation  27
---
Realisation  28
---
Realisation  29
---
Realisation  30
---
Realisation  31
---
Realisation  32
---
Realisation  33
---
Realisation  34
---
Realisation  35
---
Realisation  36
---
Realisation  37
---
Realisation  38
---
Realisation  39
---
Realisation  40
---
Realisation  41
---
Realisation  42
---
Realisation  43
---
Realisation  44
---
Realisation  45
---
Realisation  46
---
Realisation  47
---
Realisation  48
---
Realisation  49
---
Realis

---
Realisation  396
---
Realisation  397
---
Realisation  398
---
Realisation  399
---
Realisation  400
---
Realisation  401
---
Realisation  402
---
Realisation  403
---
Realisation  404
---
Realisation  405
---
Realisation  406
---
Realisation  407
---
Realisation  408
---
Realisation  409
---
Realisation  410
---
Realisation  411
---
Realisation  412
---
Realisation  413
---
Realisation  414
---
Realisation  415
---
Realisation  416
---
Realisation  417
---
Realisation  418
---
Realisation  419
---
Realisation  420
---
Realisation  421
---
Realisation  422
---
Realisation  423
---
Realisation  424
---
Realisation  425
---
Realisation  426
---
Realisation  427
---
Realisation  428
---
Realisation  429
---
Realisation  430
---
Realisation  431
---
Realisation  432
---
Realisation  433
---
Realisation  434
---
Realisation  435
---
Realisation  436
---
Realisation  437
---
Realisation  438
---
Realisation  439
---
Realisation  440
---
Realisation  441
---
Realisation  442
---
Realisati

In [24]:
# add expected points and std to data frame
all_players_params['Expected Points'] = C
all_players_params['Std Points'] = S
all_players_params['Lower 95% Confidence Points'] = np.fmax(0, C - (2 * S))
all_players_params['Upper 95% Confidence Points'] = C + (2 * S)
all_players_params['Expected Goals'] = G
all_players_params['Expected Clean Sheets'] = CS
all_players_params['Expected Minutes Played'] = MP
all_players_params['Expected Kante Bonus'] = KB

In [25]:
# save to csv
frame = all_players_params
frameref = frame[['player', 'position', 'team', 'Expected Points',
                  'Std Points', 'Lower 95% Confidence Points',
                  'Upper 95% Confidence Points', 'Expected Goals', 'Expected Clean Sheets',
                  'Expected Minutes Played', 'Expected Kante Bonus']]
frameref.to_csv("../data/expected_points_20192020.csv")

In [42]:
# extract players already in teams
leagueId = "5HrRGo92iWoHuvYdX"
gw = 9
aPlayers = all_players_params.loc[availablePlayers(leagueId, gw), :]
#aPlayers = all_players_params

player out:  RLqRteW98cgNGg4wh
 
player out:  4CpoLXXynZeBkwcML
player out:  z3J6X2p7Wd88H24Lp
player out:  TYSaxKtHXJQteNNEK
player out:  BfFKoMzF95iZw4Fy3


In [43]:
# extract best current players
frame = aPlayers
frame.sort_values(by=['Expected Points'], ascending=False)[['ID', 'player', 'position', 'team', 'Expected Points', 'Std Points', 'Expected Goals', 'Expected Clean Sheets', 'Expected Kante Bonus']]

,ID,player,position,team,Expected Points,Std Points,Expected Goals,Expected Clean Sheets,Expected Kante Bonus
60,cbjcKNRepAZPXhtB8,Andros Townsend,MID,Crystal Palace,26.088,8.899902,0.490,0.872,11.634
403,qGdNbvedLCZAiW6LF,Kenny McLean,MID,Norwich,21.376,9.814001,0.998,0.792,4.626
19,4FW6J69q72tHNSLK3,James McArthur,MID,Crystal Palace,20.752,7.161180,0.354,0.928,6.534
175,RLqRteW98cgNGg4wh,Paul Pogba,MID,Man United,20.466,10.007040,0.814,1.140,3.062
205,tEyxrisP76mHE4ynE,Dale Stephens,MID,Brighton,20.326,7.578900,0.172,1.080,8.494
119,DyHE25p5zNnScuBAB,Nathan Aké,DEF,Bournemouth,20.250,7.610223,0.520,1.190,0.000
503,oAxjP2vzbDB4zBXp6,Ibrahim Amadou,MID,Norwich,20.138,10.506139,0.538,0.790,6.506
191,duKWk786JJhQtj33F,Oriol Romeu Vidal,MID,Southampton,19.578,6.395930,0.168,0.824,8.804
157,YMijYsfSpznCceQMB,Etienne Capoue,MID,Watford,19.000,8.683317,0.210,0.720,8.468
95,EkQvDZpDQQuAiQwY7,Cheikhou Kouyaté,MID,Crystal Palace,18.912,7.337865,0.124,0.766,7.816


In [44]:
# extract best current goalkeepers
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'GKP'], :]
frame.sort_values(by=['Expected Points'], ascending=False)[['ID', 'player', 'position', 'team', 'Expected Points', 'Std Points', 'Expected Goals', 'Expected Clean Sheets', 'Expected Kante Bonus']]

,ID,player,position,team,Expected Points,Std Points,Expected Goals,Expected Clean Sheets,Expected Kante Bonus
467,LEz4Nqry3pZzALCnn,Dean Henderson,GKP,Sheffield United,15.230,4.948242,0.068,1.274,0.0
200,sQNqBiq8xBXnv2EnN,Mathew Ryan,GKP,Brighton,14.932,4.574645,0.068,1.176,0.0
28,3a2jw8KorkQMNAR9d,Lukasz Fabianski,GKP,West Ham,14.670,4.396487,0.046,1.186,0.0
56,DzSw2swgp4pRcbdRJ,Hugo Lloris,GKP,Tottenham,14.362,4.958524,0.052,1.294,0.0
174,Quxv77QEPMC8qoxoS,Ben Foster,GKP,Watford,14.130,3.989624,0.050,0.920,0.0
522,grxjnNB84xgsYyb8j,Adrián San Miguel del Castillo,GKP,Liverpool,12.532,5.588647,0.080,1.102,0.0
475,8KkA5KNqgBiE8zPMw,Jonas Lössl,GKP,Everton,9.734,5.369846,0.062,0.788,0.0
43,PTDGvsdnbvsCbeDCz,Asmir Begovic,GKP,Bournemouth,9.720,5.074800,0.038,0.792,0.0
270,hRNPDnoqix67QYQyR,Vicente Guaita,GKP,Crystal Palace,9.660,4.410941,0.034,0.670,0.0
489,X8Hi9GgeLXKb3u5vb,Aaron Ramsdale,GKP,Bournemouth,8.440,5.230526,0.046,0.628,0.0


In [45]:
# extract best current defenders
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'DEF'], :]
frame.sort_values(by=['Expected Points'], ascending=False)[['ID', 'player', 'position', 'team', 'Expected Points', 'Std Points', 'Expected Goals', 'Expected Clean Sheets', 'Expected Kante Bonus']]

,ID,player,position,team,Expected Points,Std Points,Expected Goals,Expected Clean Sheets,Expected Kante Bonus
119,DyHE25p5zNnScuBAB,Nathan Aké,DEF,Bournemouth,20.250,7.610223,0.520,1.190,0.0
410,E9xZiFKcvWqA77RGK,Jack O'Connell,DEF,Sheffield United,17.890,6.808370,0.288,1.268,0.0
421,Gcm6E4sgEW6iaH3gH,Chris Basham,DEF,Sheffield United,17.784,6.495948,0.284,1.256,0.0
412,sJukwvqQ68LTWSvwN,John Egan,DEF,Sheffield United,17.500,6.580122,0.256,1.272,0.0
472,Dhu6ECFvhPtJkmjjp,Dan Burn,DEF,Brighton,17.176,6.549277,0.292,1.154,0.0
63,rfRgQzthBzuDxLRxs,David Luiz Moreira Marinho,DEF,Arsenal,16.840,8.262106,0.482,1.132,0.0
64,rpk43BdQKtXoPwS36,Ashley Young,DEF,Man United,16.404,6.874648,0.234,1.252,0.0
393,iYsWZXKs5YriXhhs8,Jamal Lewis,DEF,Norwich,16.344,6.397317,0.268,0.964,0.0
390,an936ZBSTqKTPZjQE,Ben Godfrey,DEF,Norwich,16.170,6.592200,0.288,0.948,0.0
203,sHnZmw2yACnuTi2wM,Shane Duffy,DEF,Brighton,16.020,7.740258,0.390,0.962,0.0


In [46]:
# extract best current midfielders
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'MID'], :]
frame.sort_values(by=['Expected Points'], ascending=False)[['ID', 'player', 'position', 'team', 'Expected Points', 'Std Points', 'Expected Goals', 'Expected Clean Sheets', 'Expected Kante Bonus']]

,ID,player,position,team,Expected Points,Std Points,Expected Goals,Expected Clean Sheets,Expected Kante Bonus
60,cbjcKNRepAZPXhtB8,Andros Townsend,MID,Crystal Palace,26.088,8.899902,0.490,0.872,11.634
403,qGdNbvedLCZAiW6LF,Kenny McLean,MID,Norwich,21.376,9.814001,0.998,0.792,4.626
19,4FW6J69q72tHNSLK3,James McArthur,MID,Crystal Palace,20.752,7.161180,0.354,0.928,6.534
175,RLqRteW98cgNGg4wh,Paul Pogba,MID,Man United,20.466,10.007040,0.814,1.140,3.062
205,tEyxrisP76mHE4ynE,Dale Stephens,MID,Brighton,20.326,7.578900,0.172,1.080,8.494
503,oAxjP2vzbDB4zBXp6,Ibrahim Amadou,MID,Norwich,20.138,10.506139,0.538,0.790,6.506
191,duKWk786JJhQtj33F,Oriol Romeu Vidal,MID,Southampton,19.578,6.395930,0.168,0.824,8.804
157,YMijYsfSpznCceQMB,Etienne Capoue,MID,Watford,19.000,8.683317,0.210,0.720,8.468
95,EkQvDZpDQQuAiQwY7,Cheikhou Kouyaté,MID,Crystal Palace,18.912,7.337865,0.124,0.766,7.816
99,eGpSZSMP6Xr8jRuEt,Idrissa Gueye,MID,Everton,18.850,8.926113,0.162,0.926,9.248


In [47]:
# extract best current forwards
frame = aPlayers
frame = frame.loc[frame.index[frame['position'] == 'FWD'], :]
frame.sort_values(by=['Expected Points'], ascending=False)[['ID', 'player', 'position', 'team', 'Expected Points', 'Std Points', 'Expected Goals', 'Expected Clean Sheets', 'Expected Kante Bonus']]

,ID,player,position,team,Expected Points,Std Points,Expected Goals,Expected Clean Sheets,Expected Kante Bonus
25,6xP93deqcTwpbqH6T,Dominic Calvert-Lewin,FWD,Everton,15.766,6.952355,0.782,0.340,4.760
125,NLsEGv83GLpK6ymfW,Troy Deeney,FWD,Watford,12.134,6.583012,0.696,0.584,1.144
127,RGN6Ec5ACcFY5xgps,Romelu Lukaku,FWD,Man United,11.936,6.248192,0.862,0.956,0.942
209,KvTfLqoDib7usQpkA,Glenn Murray,FWD,Brighton,11.416,5.588107,0.996,0.378,0.550
447,rbAoKp4RtpbDvK37h,Jay Rodriguez,FWD,Burnley,11.074,5.402640,0.598,0.034,3.036
132,DivM2uKJYCCBdzNpD,Marko Arnautovic,FWD,West Ham,11.070,6.489461,0.730,0.706,0.564
529,QvNp324X2hoyoX7FQ,Aaron Connolly,FWD,Brighton,10.226,5.259175,1.030,0.000,0.682
396,GuMQfM3yHeAWhoqad,Dennis Srbeny,FWD,Norwich,9.286,6.822624,0.058,0.000,6.668
433,PvdJYKz5wdhTrMKWp,Che Adams,FWD,Southampton,8.486,3.912263,0.194,0.388,0.206
83,bmTfC26gnGLp67NAb,Andre Gray,FWD,Watford,8.172,4.464349,0.638,0.012,0.438


### Analysis of previous season - model verification

In [21]:
# create fixture list this season to predict
fixture_list_last_season = pd.read_csv("../data/prem_results_20182019.csv", header=None).values

all_players_params = pd.read_csv("../parameters/all_players_params.csv")
all_teams_params = pd.read_csv("../parameters/all_teams_params.csv", header=None)
teams = pd.read_csv("../parameters/all_teams.csv", header=None).values[:, 0]

C, S, CS, G, MP, KB = ComputeExpectedPoints(fixture_list_last_season, teams, all_players_params, all_teams_params, Niter=50,
                                        additionalstats=True)

utils.py:382: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  intercept = (all_teams_params.as_matrix())[0, :]
utils.py:383: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  mu = (all_teams_params.as_matrix())[1, :]
utils.py:384: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  a = (all_teams_params.as_matrix())[2:(len(teams) + 2), :]
utils.py:385: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  d = (all_teams_params.as_matrix())[(len(teams) + 2):((2 * len(teams)) + 2), :]


KeyboardInterrupt: 

In [ ]:
# plot with respect to last seasons points - for those players eligible
y3 = pd.read_csv("../data/players_raw_20182019.csv")
y3['full_name'] = (y3['first_name'] + ' ' + y3['second_name'])
exp_points = []
sd_points = []
true_points = []
exp_goals = []
true_goals = []
exp_cs = []
true_cs = []
exp_mp = []
true_mp = []
cr = []
for i in range(len(all_players_params.index)):
    ind = np.where(all_players_params.loc[all_players_params.index[i], 'player'] == y3['full_name'])[0]
    if (len(ind) > 0):
        exp_points.append(C[i])
        sd_points.append(S[i])
        exp_cs.append(CS[i])
        exp_mp.append(MP[i])
        exp_goals.append(G[i])
        true_points.append(y3.loc[y3.index[ind[0]], 'total_points'] - y3.loc[y3.index[ind[0]], 'bonus'])
        true_goals.append(y3.loc[y3.index[ind[0]], 'goals_scored'])
        true_mp.append(y3.loc[y3.index[ind[0]], 'minutes'])
        true_cs.append(y3.loc[y3.index[ind[0]], 'clean_sheets'])
        cr.append([((all_players_params.loc[all_players_params.index[i], 'position'] == 'FWD')),
                   ((all_players_params.loc[all_players_params.index[i], 'position'] == 'MID')),
                   ((all_players_params.loc[all_players_params.index[i], 'position'] == 'DEF'))])


exp_points = np.array(exp_points)
sd_points = np.array(sd_points)
exp_goals = np.array(exp_goals)
exp_cs = np.array(exp_cs)
exp_mp = np.array(exp_mp)
true_points = np.array(true_points)
true_goals = np.array(true_goals)
true_mp = np.array(true_mp)
true_cs = np.array(true_cs)

plot.errorbar(exp_points, true_points, xerr=2 * sd_points, marker='.', linestyle="", alpha=0.1)
plot.scatter(exp_points, true_points, color=cr, s=6)
plot.plot(true_points, true_points, linestyle="--", color='g')
plot.xlim([0, 350])
plot.ylim([0, 350])
plot.xlabel('Expected Points 2018/2019')
plot.ylabel('Actual Points 2018/2019')
plot.show()

In [ ]:
# plot with respect to last seasons clean sheets - for eligble players
plot.scatter(exp_cs, true_cs, color=cr, s=6)
plot.plot(true_cs, true_cs, linestyle="--", color='g')
plot.xlim([0, 27])
plot.ylim([0, 27])
plot.xlabel('Expected Clean Sheets 2018/2019')
plot.ylabel('Actual Clean Sheets 2018/2019')
plot.show()

In [ ]:
# plot with respect to last seasons goals - for eligble players
plot.scatter(exp_goals, true_goals, color=cr, s=6)
plot.plot(true_goals, true_goals, linestyle="--", color='g')
plot.xlim([0, 30])
plot.ylim([0, 30])
plot.xlabel('Expected Goals 2018/2019')
plot.ylabel('Actual Goals 2018/2019')
plot.show()